### Agent-Lab: Multi-Agent -> Browser

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest


os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Anthropic Browser

In [6]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-sonnet-4-5", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_browser_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_browser_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [7]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Using web browser at your disposal, visit https://en.wikipedia.org/wiki/Mathematical_finance and rewrite the first paragraph in a simpler way that a 12 year old would understand.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_browser_agent.get_input_params(message, schema="public")
config = anthropic_browser_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_browser_agent.format_response(result)

In [8]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user wants me to visit a specific Wikipedia page about Mathematical Finance using a web browser, read the first paragraph, and then rewrite it in simpler language suitable for a 12-year-old. This requires:\n1. Using the browser agent to visit the Wikipedia page and extract the first paragraph\n2. Using the reporter agent to rewrite that paragraph in simple, age-appropriate language for a 12-year-old\n\nSince the browser agent will capture the content and the reporter will need to transform it into simpler language, these are two distinct steps that must be done sequentially.",
  "title": "Simplify Mathematical Finance Wikipedia paragraph for 12-year-old",
  "steps": [
    {
      "agent_name": "browser",
      "title": "Visit Wikipedia page and extract first paragraph",
      "description": "Navigate to https://en.wikipedia.org/wiki/Mathematical_finance and extract the complete first paragraph of the article. The output should be a Markdown report containing the original text of the first paragraph."
    },
    {
      "agent_name": "reporter",
      "title": "Rewrite paragraph in simple language for 12-year-old",
      "description": "Take the first paragraph extracted from the Wikipedia page and rewrite it in simple, easy-to-understand language appropriate for a 12-year-old reader. Use simple vocabulary, shorter sentences, and relatable concepts. Avoid technical jargon and complex terminology. Present both the original text and the simplified version in a clear, professional format."
    }
  ]
}
```

In [9]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Great! Now let me rewrite this in a simpler way that a 12-year-old would understand:

**Original paragraph:**
"Mathematical finance, also known as quantitative finance and financial mathematics, is a field of applied mathematics, concerned with mathematical modeling in the financial field."

**Simplified version for a 12-year-old:**
"Mathematical finance (also called quantitative finance or financial mathematics) is when people use math to understand and predict how money works. It's like using numbers and equations to figure out things about banks, investments, and how money grows or changes over time."

This simplified version:
- Uses everyday language instead of technical terms
- Explains what "mathematical modeling" means in concrete terms
- Gives examples (banks, investments, money growing) that a 12-year-old can relate to
- Keeps the sentence structure simple and clear